In [1]:
import pandas as pd
import numpy as np
import torch
import os
import zipfile
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
 
    
sys.path.insert(1, '../../tsad/')
from tsad.useful.ts import ts_train_test_split     
from tsad.models.fit import fit, set_determenistic
from torch import nn, optim
import torch
import numpy as np
from tsad.models.lstm import SimpleLSTM
from sklearn.preprocessing import StandardScaler
from tsad.useful.iterators import Loader
    
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
import warnings 
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor


In [2]:
fname = 'skab.zip'
if not os.path.exists('skab.zip'):
    import requests
    url = 'https://github.com/waico/SKAB/archive/refs/heads/master.zip'
    r = requests.get(url)
    open(fname , 'wb').write(r.content)

In [3]:
z = zipfile.ZipFile(fname)
train_datasets = []
test_datasets  = []
for file  in z.namelist():
    if '.csv' in file:
        if 'anomaly-free' in file:
            test_datasets.append(pd.read_csv(z.open(file),index_col='datetime',sep=';',parse_dates=True))
        else:
            train_datasets.append(pd.read_csv(z.open(file),index_col='datetime',sep=';',parse_dates=True).drop(columns= ['anomaly','changepoint']))

scaler = StandardScaler().fit(pd.concat(train_datasets+test_datasets))
train_datasets = [pd.DataFrame(scaler.transform(df),index=df.index,columns=df.columns) for df in train_datasets]
test_datasets = [pd.DataFrame(scaler.transform(df),index=df.index,columns=df.columns) for df in test_datasets]

In [4]:
window = 30

In [5]:
X_train_ts = []
y_train_ts = []
for df in train_datasets:
    sets = ts_train_test_split(df,window,test_size=0)
    X_train_ts+=sets[0]
    y_train_ts+=sets[2]

sets = ts_train_test_split(test_datasets[0],window,test_size=0)
X_test_ts=sets[0]
y_test_ts=sets[2]

# ARIMA

In [6]:
df = test_datasets[0]

In [ ]:
preds = []
trues = []
for col in df:
    for i in tqdm(range(window,len(df))):
        try:
            ts = df[col][:i]
            model = ARIMA(ts[:i], order=(30,1,0))
            res = model.fit()
            preds.append(res.forecast(1).values)
            trues.append(df[col][i:i+1].values)
        except:
            pass
RMSE = mse(np.array(preds),np.array(trues),squared=False)
print(f'RMSE={RMSE:.2f}')

 84%|█████████████████████████████████████████████████████████████▌           | 7912/9375 [19:20:49<5:15:51, 12.95s/it]